In [172]:
import six
from itertools import product

In [173]:
DATA = """
.#.
..#
###
""".strip()

In [174]:
class HyperSpace:
    
    """
    Utility to solve AoC's 17th puzzle of 2020; it stores
    a representation of a D-dimensional space populated by
    active and inactive cubes, which at any time step update
    their state based on a set of given rules.
    """
    
    def __init__(self, layer, numberOfDimensions=3):
        
        """
        Initialises the hyperspace with a single
        2-dimensional slice.
        
        :param layer: list(list(str,))
        :param numberOfDimensions: int
        """
        
        self.numberOfDimensions = numberOfDimensions
        
        # The internal data is stored as a mapping of coordinates
        # to whether they are active; this was chosen over an array-based
        # or tensor-like structure arbitrarily, but higher performance
        # is certainly achievable adopting one of the above;
        
        self.layout = {}
        for y, row in enumerate(layer.splitlines()):
            for x, value in enumerate(row):
                loc = [x, y] + [0 for _ in range(numberOfDimensions - 2)]
                loc = tuple(loc)
                self.layout[loc] = value == '#'
                
    def read(self, *coordinates):
        
        """
        Returns the value found at the given coordinates.
        
        :param coordinates: (int,)
        :return: bool
        """
        
        # Given the hyperspace is infinite, if the coordinates
        # are not found in the layout, the default inactive
        # value is returned;
        
        return self.layout.get(coordinates, False)
    
    def flip(self, *coordinates):
        
        """
        Flips the state of a coordinate in the hyperspace.
        
        :param coordinates: (int,)
        """
        
        self.layout[coordinates] = not self.layout[coordinates]
                
    def neighbours(self, *coordinates):
        
        """
        Yields the coordinates and states of all the neighbours
        of the given coordinate.
        
        :param coordinates: (int,)
        :yield: neighbourCoordinates:tuple(int,), isActive:bool
        """
        
        # To find the neighbours, the (Cartesian) product of the
        # unit displacement in all dimension is considered, but
        # excluding the null displacement as it would consider
        # the given coordinates as a neeighbour;
        
        for delta in product(*[(-1, 0, +1) for _ in range(self.numberOfDimensions)]):
            if all(v == 0 for v in delta): continue
            neighbourLoc = []
            for x, dx in zip(coordinates, delta): neighbourLoc.append(x + dx)
            neighbourLoc = tuple(neighbourLoc)
            yield neighbourLoc, self.read(*neighbourLoc)
            
    def takeSteps(self, n):
        """
        Performs a timestep in the hyperspace and updates the
        state of every cube based on the given rules.
        """
        for _ in range(n): self._step()
        return self
    
    def _step(self):
        
        # First, all the coordinates (out of the infinitely
        # available locations) which might need updating
        # are found; these are all the explicitly encountered
        # locations so far with the addition (set union) of all
        # their neighbours; this is because, as per the given
        # rules, an inactive cube (which is the default in the
        # hyperspace) will only update if it has exactly three
        # active neighbours, hence only direct neighbours of
        # active known cubes are considered. Once found all
        # the new cubes, explicitly write them in the layout;
        
        toCreate = set()
        for coordinates, v in six.iteritems(self.layout):
            if not v: continue
            neighbours = list(self.neighbours(*coordinates))
            for coordinates, _ in neighbours:
                if coordinates not in self.layout:
                    toCreate.add(coordinates)
        for coordinates in toCreate:
            self.layout[coordinates] = False
            
        # For all known locations, determine whether their
        # values needs updating and store the result in a
        # temporary set (this allows updating 'simultaneously');
        
        toFlip = set()
        for coordinates, v in six.iteritems(self.layout):
            neighbours = list(self.neighbours(*coordinates))
            numberOfActiveNeighbours = sum(isActive for _, isActive in neighbours)
            if v and numberOfActiveNeighbours not in (2, 3): toFlip.add(coordinates)
            elif not v and numberOfActiveNeighbours == 3: toFlip.add(coordinates)

        for coordinates in toFlip: self.flip(*coordinates)
            
    def count(self):
        """
        Retrieves how many active cubes there are in the hyperspace.
        
        :return: int
        """
        return sum(six.itervalues(self.layout))

In [175]:
HyperSpace(DATA, numberOfDimensions=3).takeSteps(6).count()

112

In [176]:
HyperSpace(DATA, numberOfDimensions=4).takeSteps(6).count()

848